In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
Асинхронный LinkedIn Job Scraper с:
- Ротацией User-Agent
- Тремя фильтрами по дате (1д, 7д, 30д)
- Сбросом checkpoint-а, чтобы при каждом запуске генерировать все ссылки
- Поддержкой Jupyter (nest_asyncio) и консоли
"""

import asyncio
try:
    import nest_asyncio
    nest_asyncio.apply()
except ImportError:
    pass
from datetime import datetime
from bs4 import BeautifulSoup
import re
import aiohttp
import random
import pandas as pd
import json
import logging
import os
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

# ------------------------- Конфигурация -------------------------

USER_AGENTS = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 Chrome/113.0.0.0 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:112.0) Gecko/20100101 Firefox/112.0",
    "Mozilla/5.0 (iPhone; CPU iPhone OS 16_0 like Mac OS X) AppleWebKit/605.1.15 Version/16.0 Mobile/15E148 Safari/604.1",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 Chrome/113.0.0.0 Edg/113.0.0.0"
]

TIME_PARAMS = ["r86400", "r604800", "r2592000"]  # 1d, 7d, 30d 
GERMANY_CITIES = [
    "Berlin", "Hamburg", "Munich", "Frankfurt", "Cologne", "Stuttgart",
    "Duesseldorf", "Leipzig", "Dresden", "Hanover", "Nuremberg",
    "Bremen", "Essen", "Dortmund"
]
KEYWORDS = [
    "QA Manual Tester", "Manual Tester", "Software Tester",
    "QA Engineer", "Test Engineer", "Software Test Analyst",
    "Manual QA Engineer", "Functional Tester", "Test Analyst"
]

CONCURRENCY_LIMIT = 20
RETRIES = 3
TIMEOUT = 15       # секунд до таймаута
BATCH_SIZE = 200
MAX_PAGES = 40     # 40 страниц × 25 вакансий = до 1000 URL на фильтр
  # сколько вакансий брать на тест

OUTPUT_CSV = 'raw_qa_manual_tester_germany.csv'
CHECKPOINT_FILE = 'checkpoint.json'
ERROR_LOG = 'errors.log'

logging.basicConfig(
    filename=ERROR_LOG,
    filemode='a',
    level=logging.WARNING,
    format='%(asctime)s %(levelname)s %(message)s'
)

# ------------------------- Утилиты -------------------------

def sanitize(text: str) -> str:
    return text.replace(" ", "%20") \
               .replace("ü", "ue") \
               .replace("ö", "oe") \
               .replace("ä", "ae") \
               .replace("ß", "ss")

def run_coro(coro):
    """
    Безопасный запуск корутины в Jupyter или в консоли.
    """
    try:
        return asyncio.run(coro)
    except RuntimeError:
        loop = asyncio.get_event_loop()
        return loop.run_until_complete(coro)

async def fetch(session: aiohttp.ClientSession, url: str) -> str:
    """GET-запрос с retry и ротацией User-Agent."""
    for attempt in range(1, RETRIES + 1):
        try:
            headers = {"User-Agent": random.choice(USER_AGENTS)}
            async with session.get(url, headers=headers, timeout=TIMEOUT) as resp:
                resp.raise_for_status()
                return await resp.text()
        except Exception as e:
            logging.warning(f"[{attempt}/{RETRIES}] {e} @ {url}")
            await asyncio.sleep(2 ** attempt)
    logging.error(f"All retries failed for {url}")
    return ""

def parse_job_cards(html: str, kw: str, city: str) -> list:
    """Извлекает из страницы поиска карточки вакансий."""
    soup = BeautifulSoup(html, 'html.parser')
    cards = soup.find_all('div', class_='base-card')
    jobs = []
    for card in cards:
        link_el = card.select_one('a.base-card__full-link')
        jobs.append({
            'Title':    (card.select_one('h3.base-search-card__title') or '').get_text(strip=True),
            'Company':  (card.select_one('h4.base-search-card__subtitle') or '').get_text(strip=True),
            'Location': (card.select_one('span.job-search-card__location') or '').get_text(strip=True),
            'Link':     link_el['href'] if link_el and link_el.has_attr('href') else '',
            'Keyword':  kw,
            'City':     city
        })
    return jobs

def parse_description(html: str) -> str:
    """Извлекает описание вакансии из её страницы."""
    soup = BeautifulSoup(html, 'html.parser')
    selectors = [
        ('div', 'show-more-less-html__markup'),
        ('section', 'description'),
        ('div', 'description'),
    ]
    for tag, cls in selectors:
        el = soup.find(tag, class_=cls)
        if el:
            return el.get_text(separator=' ', strip=True)
    return "Error"

def save_batch(df: pd.DataFrame, first_batch: bool):
    """Сохраняет DataFrame в CSV; пропускает пустые пачки."""
    if df.empty:
        print("⚠️ Empty batch — skipping save.")
        return
    mode = 'w' if first_batch else 'a'
    header = first_batch
    df.to_csv(OUTPUT_CSV, index=False, mode=mode, header=header)
    print(f"✔️ Saved {len(df)} jobs to {OUTPUT_CSV}")

# ------------------------- Этап 1: сбор ссылок -------------------------

async def gather_search_pages() -> list:
    urls = []
    # Генерация всех URL
    for tp in TIME_PARAMS:
        for city in GERMANY_CITIES:
            for kw in KEYWORDS:
                for page in range(MAX_PAGES):
                    start = page * 25
                    url = (
                        "https://www.linkedin.com/jobs/search/"
                        f"?keywords={sanitize(kw)}"
                        f"&location={sanitize(city)}"
                        f"&f_TPR={tp}"
                        f"&start={start}"
                    )
                    urls.append((url, kw, city))

    print(f"🔗 Total URLs to fetch: {len(urls)}")  # Должно быть >> 12

    connector = aiohttp.TCPConnector(limit=CONCURRENCY_LIMIT)
    async with aiohttp.ClientSession(connector=connector) as session:
        sem = asyncio.Semaphore(CONCURRENCY_LIMIT)
        jobs = []

        async def worker(u, kw, city):
            async with sem:
                html = await fetch(session, u)
                if not html:
                    return
                cards = await asyncio.get_event_loop().run_in_executor(
                    None, parse_job_cards, html, kw, city
                )
                jobs.extend(cards)

        tasks = [worker(u, kw, city) for u, kw, city in urls]
        for f in tqdm(asyncio.as_completed(tasks), total=len(tasks), desc="Gathering links"):
            await f

    # Убираем дубликаты по URL
    unique = {j['Link']: j for j in jobs if j['Link']}
    return list(unique.values())

# ------------------------- Этап 2: сбор описаний -------------------------

async def gather_descriptions(jobs: list):
    connector = aiohttp.TCPConnector(limit=CONCURRENCY_LIMIT)
    executor = ThreadPoolExecutor(max_workers=CONCURRENCY_LIMIT)

    async with aiohttp.ClientSession(connector=connector) as session:
        sem = asyncio.Semaphore(CONCURRENCY_LIMIT)
        first_batch = True

        # Разбиваем список на батчи
        for start in range(0, len(jobs), BATCH_SIZE):
            end = min(start + BATCH_SIZE, len(jobs))
            batch = jobs[start:end]

            # Одна ворк-функция для каждого job
            async def worker(job):
                async with sem:
                    html = await fetch(session, job['Link'])
                    job['ParsedDate'] = datetime.now().strftime('%Y-%m-%d')

                    if not html:
                        job.update({
                            'Description': 'Error',
                            'PostedDate': None,
                            'EmploymentType': None,
                            'SeniorityLevel': None,
                            'Skills': None,
                            'Industries': None,
                            'JobFunction': None,
                            'RemoteStatus': None,
                            'Salary': None
                        })
                        return

                    soup = BeautifulSoup(html, 'html.parser')

                    # 1) JSON-LD
                    ld = {}
                    tag = soup.find('script', {'type': 'application/ld+json'})
                    if tag:
                        try:
                            ld = json.loads(tag.string)
                        except:
                            ld = {}

                    # 2) Guest API
                    api_data = {}
                    if not ld.get('skills') or not ld.get('experienceRequirements'):
                        m = re.search(r'jobId=(\d+)', job['Link'])
                        if m:
                            jid = m.group(1)
                            url = f'https://www.linkedin.com/jobs-guest/jobs/api/jobDetails?jobId={jid}'
                            resp = await session.get(url)
                            if resp.status == 200:
                                try:
                                    api_data = await resp.json()
                                except:
                                    api_data = {}

                    # 3) Задезкрипшн
                    job['Description'] = await asyncio.get_event_loop() \
                        .run_in_executor(executor, parse_description, html)

                    # 4) Наполняем поля
                    job['PostedDate'] = extract_posted_date(soup)

                    job['EmploymentType'] = (
                        ld.get('employmentType')
                        or api_data.get('employmentType')
                        or extract_employment_type(soup)
                    )

                    job['SeniorityLevel'] = (
                        ld.get('experienceRequirements')
                        or api_data.get('experienceRequirements')
                        or extract_seniority_level(soup)
                    )

                    # Skills
                    skills = ld.get('skills') or api_data.get('skills')
                    if isinstance(skills, list):
                        job['Skills'] = ', '.join(skills)
                    else:
                        job['Skills'] = skills or extract_skills(soup)

                    job['Industries'] = (
                        ld.get('industry')
                        or api_data.get('industry')
                        or extract_industries(soup)
                    )

                    job['JobFunction'] = (
                        ld.get('jobFunction')
                        or api_data.get('jobFunction')
                        or extract_job_function(soup)
                    )

                    job['RemoteStatus'] = (
                        ld.get('workRemoteInstructions')
                        or api_data.get('workRemoteInstructions')
                        or extract_remote_status(soup)
                    )

                    # Salary
                    salary = (ld.get('baseSalary') or {}).get('value') \
                             or api_data.get('salary')
                    job['Salary'] = salary or extract_salary(soup)

            # Собираем и запускаем корутины
            tasks = [worker(j) for j in batch]
            for f in tqdm(
                asyncio.as_completed(tasks),
                total=len(tasks),
                desc=f"Fetching desc {start+1}-{end}"
            ):
                await f

            # Сохраняем батч в CSV
            df = pd.DataFrame(batch)
            save_batch(df, first_batch)
            first_batch = False

        executor.shutdown()

def extract_seniority_level(soup):
    text = soup.get_text().lower()
    if 'junior' in text:
        return 'Junior'
    elif 'mid' in text or 'mittel' in text:
        return 'Mid'
    elif 'senior' in text or 'leitung' in text:
        return 'Senior'
    elif re.search(r'\b(6|7|8)\s*\+\s*jahre\b', text):
        return 'Senior'
    return 'Unknown'

def extract_skills(soup):
    text = soup.get_text().lower()
    keywords = ['python', 'selenium', 'jira', 'sql', 'sap', 'test automation', 'manual testing', 'erp']
    found = [kw.capitalize() for kw in keywords if kw in text]
    return ', '.join(found) if found else None

def extract_employment_type(soup):
    text = soup.get_text().lower()
    if 'vollzeit' in text or 'full-time' in text:
        return 'Full-time'
    elif 'teilzeit' in text or 'part-time' in text:
        return 'Part-time'
    return 'Unknown'

def extract_remote_status(soup):
    text = soup.get_text().lower()
    if 'remote' in text:
        return 'Remote'
    elif 'hybrid' in text:
        return 'Hybrid'
    elif 'vor ort' in text or 'on-site' in text:
        return 'On-site'
    return 'Unknown'

def extract_salary(soup):
    text = soup.get_text()
    matches = re.findall(r'€\s*\d+[.,]?\d*', text)
    return matches[0] if matches else None

def extract_posted_date(soup):
    tag = soup.find('span', class_='posted-time-ago__text')
    return tag.text.strip() if tag else None

def extract_industries(soup):
    text = soup.get_text().lower()
    for keyword in ['it', 'software', 'telekommunikation', 'energie', 'logistik']:
        if keyword in text:
            return keyword.capitalize()
    return None

def extract_job_function(soup):
    text = soup.get_text().lower()
    for keyword in ['test', 'entwicklung', 'analyse', 'support']:
        if keyword in text:
            return keyword.capitalize()
    return None


# ------------------------- Главная функция -------------------------

def main():
    # Сбрасываем старые данные и checkpoint, чтобы стартовать с чистого листа
    if os.path.exists(CHECKPOINT_FILE):
        os.remove(CHECKPOINT_FILE)
    if os.path.exists(OUTPUT_CSV):
        os.remove(OUTPUT_CSV)

    print("🔍 Gathering job links...")
    jobs = run_coro(gather_search_pages())
    print(f"✅ Collected {len(jobs)} unique job links.")

    print("📝 Fetching job descriptions...")
    run_coro(gather_descriptions(jobs))

    print("🎉 All done! CSV заполнен данными.")

if __name__ == "__main__":
    main()


🔍 Gathering job links...
🔗 Total URLs to fetch: 10080


Gathering links: 100%|█████████████████████████████████████████████████████████| 10080/10080 [2:02:16<00:00,  1.37it/s]


✅ Collected 3479 unique job links.
📝 Fetching job descriptions...


Fetching desc 1-200: 100%|███████████████████████████████████████████████████████████| 200/200 [02:29<00:00,  1.34it/s]


✔️ Saved 200 jobs to raw_qa_manual_tester_germany.csv


...

✔️ Saved 200 jobs to raw_qa_manual_tester_germany.csv


Fetching desc 3201-3400: 100%|███████████████████████████████████████████████████████| 200/200 [02:28<00:00,  1.34it/s]


✔️ Saved 200 jobs to raw_qa_manual_tester_germany.csv


Fetching desc 3401-3479: 100%|█████████████████████████████████████████████████████████| 79/79 [01:06<00:00,  1.20it/s]

✔️ Saved 79 jobs to raw_qa_manual_tester_germany.csv
🎉 All done! CSV заполнен данными.
